In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, TimeDistributed, Flatten, BatchNormalization
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

# 데이터셋 불러오기
file_path = 'C:\\Users\\co279\\mp_statcast.csv'
data = pd.read_csv(file_path)

# Drop unnecessary columns
data = data.drop(columns=['last_name, first_name'])

# Fill missing values with 0
data = data.fillna(0)

if 'pitch_hand' in data:
    data = pd.get_dummies(data, columns=['pitch_hand'], drop_first=True)

# 2020, 2021, 2022년에 모두 존재하는 player_id 추출
data_2020 = data[data['year'] == 2020]
data_2021 = data[data['year'] == 2021]
data_2022 = data[data['year'] == 2022]
data_2023 = data[data['year'] == 2023]

player_ids_2020 = set(data_2020['player_id'].unique())
player_ids_2021 = set(data_2021['player_id'].unique())
player_ids_2022 = set(data_2022['player_id'].unique())
player_ids_2023 = set(data_2023['player_id'].unique())

common_player_ids = player_ids_2020 & player_ids_2021 & player_ids_2022 & player_ids_2023

# Extract data for common player_ids
common_data = data[data['player_id'].isin(common_player_ids)]

# Extract data for the years 2020, 2021, and 2022
final = common_data[common_data['year'].isin([2020, 2021, 2022])]
final = final.sort_values(by=['player_id', 'year'])
# 필요한 컬럼 선택 (year 제외)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# 독립변수와 종속변수 분리
X = final[features].values
y = final[target].values

# 데이터 스케일링
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# 시계열 데이터 형태로 변환
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 3  # 시퀀스 길이 설정
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# 학습 데이터와 전체 데이터를 동일하게 설정
X_train, y_train = X_seq, y_seq

In [2]:
# LSTM_A

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_A
    model_A = Sequential()
    model_A.add(LSTM(128, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_A.add(LSTM(128, return_sequences=True))
    model_A.add(Dropout(rate=0.5))
    model_A.add(Flatten())
    model_A.add(Dense(1024, activation="relu"))
    model_A.add(Dropout(rate=0.5))
    model_A.add(Dense(1, activation='relu'))

    # 컴파일
    adam = optimizers.Adam(learning_rate=0.001)
    model_A.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # 조기 종료 콜백
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # 모델 학습
    history_A = model_A.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # 2023년 데이터 필터링
    data_23 = common_data[common_data['year'] == 2023]

    # 2023년 데이터 스케일링
    X_2023_scaled = scaler_X.transform(data_23[features].values)

    # 시계열 데이터 형태로 변환 (2022년, 2021년, 2020년 데이터를 사용하여 2023년 예측)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2023_seq = create_sequences_for_prediction(X_2023_scaled, seq_length)

    # 2023년 데이터 예측
    y_pred_scaled_A = model_A.predict(X_2023_seq)

    # 스케일 복원
    y_pred_A = scaler_y.inverse_transform(y_pred_scaled_A)

    # 실제 값과 예측 값 비교를 위해 실제 2023년 p_era 값도 복원
    y_test_actual = data_23[target].values

    # RMSE 계산
    rmse_A = np.sqrt(mean_squared_error(y_test_actual, y_pred_A))
    rmse_list.append(rmse_A)

    # MAE 계산
    mae_A = mean_absolute_error(y_test_actual, y_pred_A)
    mae_list.append(mae_A)
    
    # MAPE 계산
    mape_A = mean_absolute_percentage_error(y_test_actual, y_pred_A)
    mape_list.append(mape_A)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_A}, MAE: {mae_A}, MAPE: {mape_A}')

# 평균 RMSE, MAE, MAPE 계산
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
3/3 [==============================] - 22s 2s/step - loss: 0.0728 - accuracy: 0.0000e+00 - val_loss: 0.0556 - val_accuracy: 0.0222
Epoch 2/500
3/3 [==============================] - 0s 61ms/step - loss: 0.0332 - accuracy: 0.0000e+00 - val_loss: 0.0273 - val_accuracy: 0.0222
Epoch 3/500
3/3 [==============================] - 0s 64ms/step - loss: 0.0333 - accuracy: 0.0000e+00 - val_loss: 0.0230 - val_accuracy: 0.0222
Epoch 4/500
3/3 [==============================] - 0s 70ms/step - loss: 0.0266 - accuracy: 0.0000e+00 - val_loss: 0.0291 - val_accuracy: 0.0222
Epoch 5/500
3/3 [==============================] - 0s 69ms/step - loss: 0.0265 - accuracy: 0.0000e+00 - val_loss: 0.0220 - val_accuracy: 0.0222
Epoch 6/500
3/3 [==============================] - 0s 76ms/step - loss: 0.0251 - accuracy: 0.0000e+00 - val_loss: 0.0218 - val_accuracy: 0.0222
Epoch 7/500
3/3 [==============================] - 0s 75ms/step - loss: 0.0225 - accuracy: 0.0000e+00 - val_loss: 0.0233 - val_accuracy: 

In [3]:
# LSTM_B

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_B
    model_B = Sequential()
    model_B.add(LSTM(32, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_B.add(LSTM(16, return_sequences=True))
    model_B.add(LSTM(8, return_sequences=True))
    model_B.add(Dropout(rate=0.5))
    model_B.add(Flatten())
    model_B.add(Dense(512, activation="relu"))
    model_B.add(Dropout(rate=0.5))
    model_B.add(Dense(64, activation="relu"))
    model_B.add(Dense(1, activation='relu'))

    # 컴파일
    adam = optimizers.Adam(learning_rate=0.001)
    model_B.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # 조기 종료 콜백
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # 모델 학습
    history_B = model_B.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # 2023년 데이터 필터링
    data_23 = common_data[common_data['year'] == 2023]

    # 2023년 데이터 스케일링
    X_2023_scaled = scaler_X.transform(data_23[features].values)

    # 시계열 데이터 형태로 변환 (2022년, 2021년, 2020년 데이터를 사용하여 2023년 예측)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2023_seq = create_sequences_for_prediction(X_2023_scaled, seq_length)

    # 2023년 데이터 예측
    y_pred_scaled_B = model_B.predict(X_2023_seq)

    # 스케일 복원
    y_pred_B = scaler_y.inverse_transform(y_pred_scaled_B)

    # 실제 값과 예측 값 비교를 위해 실제 2023년 p_era 값도 복원
    y_test_actual = data_23[target].values

    # RMSE 계산
    rmse_B = np.sqrt(mean_squared_error(y_test_actual, y_pred_B))
    rmse_list.append(rmse_B)

    # MAE 계산
    mae_B = mean_absolute_error(y_test_actual, y_pred_B)
    mae_list.append(mae_B)
    
    # MAPE 계산
    mape_B = mean_absolute_percentage_error(y_test_actual, y_pred_B)
    mape_list.append(mape_B)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_B}, MAE: {mae_B}, MAPE: {mape_B}')

# 평균 RMSE, MAE, MAPE 계산
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
3/3 [==============================] - 31s 3s/step - loss: 0.1180 - accuracy: 0.0000e+00 - val_loss: 0.0723 - val_accuracy: 0.0222
Epoch 2/500
3/3 [==============================] - 0s 59ms/step - loss: 0.0819 - accuracy: 0.0000e+00 - val_loss: 0.0396 - val_accuracy: 0.0222
Epoch 3/500
3/3 [==============================] - 0s 61ms/step - loss: 0.0442 - accuracy: 0.0000e+00 - val_loss: 0.0223 - val_accuracy: 0.0222
Epoch 4/500
3/3 [==============================] - 0s 53ms/step - loss: 0.0316 - accuracy: 0.0000e+00 - val_loss: 0.0368 - val_accuracy: 0.0222
Epoch 5/500
3/3 [==============================] - 0s 60ms/step - loss: 0.0370 - accuracy: 0.0000e+00 - val_loss: 0.0286 - val_accuracy: 0.0222
Epoch 6/500
3/3 [==============================] - 0s 66ms/step - loss: 0.0294 - accuracy: 0.0000e+00 - val_loss: 0.0223 - val_accuracy: 0.0222
Epoch 7/500
3/3 [==============================] - 0s 62ms/step - loss: 0.0260 - accuracy: 0.0000e+00 - val_loss: 0.0222 - val_accuracy: 

In [4]:
# LSTM_C

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_C
    model_C = Sequential()
    model_C.add(LSTM(32, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_C.add(LSTM(32, return_sequences=True))
    model_C.add(BatchNormalization())
    model_C.add(Flatten())
    model_C.add(Dense(512, activation="relu"))
    model_C.add(BatchNormalization())
    model_C.add(Dense(64, activation="relu"))
    model_C.add(Dense(1, activation='relu'))

    # 컴파일
    adam = optimizers.Adam(learning_rate=0.001)
    model_C.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # 조기 종료 콜백
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # 모델 학습
    history_C = model_C.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # 2023년 데이터 필터링
    data_23 = common_data[common_data['year'] == 2023]

    # 2023년 데이터 스케일링
    X_2023_scaled = scaler_X.transform(data_23[features].values)

    # 시계열 데이터 형태로 변환 (2022년, 2021년, 2020년 데이터를 사용하여 2023년 예측)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2023_seq = create_sequences_for_prediction(X_2023_scaled, seq_length)

    # 2023년 데이터 예측
    y_pred_scaled_C = model_C.predict(X_2023_seq)

    # 스케일 복원
    y_pred_C = scaler_y.inverse_transform(y_pred_scaled_C)

    # 실제 값과 예측 값 비교를 위해 실제 2023년 p_era 값도 복원
    y_test_actual = data_23[target].values

    # RMSE 계산
    rmse_C = np.sqrt(mean_squared_error(y_test_actual, y_pred_C))
    rmse_list.append(rmse_C)

    # MAE 계산
    mae_C = mean_absolute_error(y_test_actual, y_pred_C)
    mae_list.append(mae_C)
    
    # MAPE 계산
    mape_C = mean_absolute_percentage_error(y_test_actual, y_pred_C)
    mape_list.append(mape_C)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_C}, MAE: {mae_C}, MAPE: {mape_C}')

# 평균 RMSE, MAE, MAPE 계산
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
3/3 [==============================] - 19s 2s/step - loss: 0.2303 - accuracy: 0.0000e+00 - val_loss: 0.0976 - val_accuracy: 0.0222
Epoch 2/500
3/3 [==============================] - 0s 40ms/step - loss: 0.1289 - accuracy: 0.0000e+00 - val_loss: 0.0976 - val_accuracy: 0.0222
Epoch 3/500
3/3 [==============================] - 0s 42ms/step - loss: 0.1150 - accuracy: 0.0000e+00 - val_loss: 0.0976 - val_accuracy: 0.0222
Epoch 4/500
3/3 [==============================] - 0s 39ms/step - loss: 0.1134 - accuracy: 0.0056 - val_loss: 0.0976 - val_accuracy: 0.0222
Epoch 5/500
3/3 [==============================] - 0s 40ms/step - loss: 0.1099 - accuracy: 0.0056 - val_loss: 0.0976 - val_accuracy: 0.0222
Epoch 6/500
3/3 [==============================] - 0s 37ms/step - loss: 0.1103 - accuracy: 0.0056 - val_loss: 0.0976 - val_accuracy: 0.0222
Epoch 7/500
3/3 [==============================] - 0s 39ms/step - loss: 0.1133 - accuracy: 0.0056 - val_loss: 0.0976 - val_accuracy: 0.0222
Epoch 8/5

In [5]:
# LSTM_D

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_D
    model_D = Sequential()
    model_D.add(Masking(mask_value=0.0, input_shape=(seq_length, X_train.shape[2])))
    model_D.add(LSTM(64, return_sequences=True))
    model_D.add(LSTM(64, return_sequences=True))
    model_D.add(Dropout(rate=0.5))
    model_D.add(TimeDistributed(Dense(10)))
    model_D.add(Flatten())
    model_D.add(Dense(512, activation="relu"))
    model_D.add(Dropout(rate=0.5))
    model_D.add(Dense(64, activation="relu"))
    model_D.add(Dense(1, activation='relu'))

    # 컴파일
    adam = optimizers.Adam(learning_rate=0.001)
    model_D.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # 조기 종료 콜백
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # 모델 학습
    history_D = model_D.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # 2023년 데이터 필터링
    data_23 = common_data[common_data['year'] == 2023]

    # 2023년 데이터 스케일링
    X_2023_scaled = scaler_X.transform(data_23[features].values)

    # 시계열 데이터 형태로 변환 (2022년, 2021년, 2020년 데이터를 사용하여 2023년 예측)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2023_seq = create_sequences_for_prediction(X_2023_scaled, seq_length)

    # 2023년 데이터 예측
    y_pred_scaled_D = model_D.predict(X_2023_seq)

    # 스케일 복원
    y_pred_D = scaler_y.inverse_transform(y_pred_scaled_D)

    # 실제 값과 예측 값 비교를 위해 실제 2023년 p_era 값도 복원
    y_test_actual = data_23[target].values

    # RMSE 계산
    rmse_D = np.sqrt(mean_squared_error(y_test_actual, y_pred_D))
    rmse_list.append(rmse_D)

    # MAE 계산
    mae_D = mean_absolute_error(y_test_actual, y_pred_D)
    mae_list.append(mae_D)
    
    # MAPE 계산
    mape_D = mean_absolute_percentage_error(y_test_actual, y_pred_D)
    mape_list.append(mape_D)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_D}, MAE: {mae_D}, MAPE: {mape_D}')

# 평균 RMSE, MAE, MAPE 계산
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
3/3 [==============================] - 46s 6s/step - loss: 0.1007 - accuracy: 0.0000e+00 - val_loss: 0.0335 - val_accuracy: 0.0222
Epoch 2/500
3/3 [==============================] - 0s 57ms/step - loss: 0.0377 - accuracy: 0.0000e+00 - val_loss: 0.0491 - val_accuracy: 0.0222
Epoch 3/500
3/3 [==============================] - 0s 49ms/step - loss: 0.0393 - accuracy: 0.0000e+00 - val_loss: 0.0246 - val_accuracy: 0.0222
Epoch 4/500
3/3 [==============================] - 0s 59ms/step - loss: 0.0292 - accuracy: 0.0000e+00 - val_loss: 0.0248 - val_accuracy: 0.0222
Epoch 5/500
3/3 [==============================] - 0s 71ms/step - loss: 0.0297 - accuracy: 0.0000e+00 - val_loss: 0.0239 - val_accuracy: 0.0222
Epoch 6/500
3/3 [==============================] - 0s 60ms/step - loss: 0.0295 - accuracy: 0.0000e+00 - val_loss: 0.0233 - val_accuracy: 0.0222
Epoch 7/500
3/3 [==============================] - 0s 52ms/step - loss: 0.0265 - accuracy: 0.0000e+00 - val_loss: 0.0271 - val_accuracy: 

In [6]:
# LSTM_E

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_E
    model_E = Sequential()
    model_E.add(LSTM(32, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_E.add(LSTM(32, return_sequences=True))
    model_E.add(Dropout(rate=0.5))
    model_E.add(Flatten())
    model_E.add(Dense(512, activation="relu"))
    model_E.add(Dropout(rate=0.5))
    model_E.add(Dense(64, activation="relu"))
    model_E.add(Dense(1, activation='relu'))

    # 컴파일
    adam = optimizers.Adam(learning_rate=0.001)
    model_E.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # 조기 종료 콜백
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # 모델 학습
    history_E = model_E.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # 2023년 데이터 필터링
    data_23 = common_data[common_data['year'] == 2023]

    # 2023년 데이터 스케일링
    X_2023_scaled = scaler_X.transform(data_23[features].values)

    # 시계열 데이터 형태로 변환 (2022년, 2021년, 2020년 데이터를 사용하여 2023년 예측)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2023_seq = create_sequences_for_prediction(X_2023_scaled, seq_length)

    # 2023년 데이터 예측
    y_pred_scaled_E = model_E.predict(X_2023_seq)

    # 스케일 복원
    y_pred_E = scaler_y.inverse_transform(y_pred_scaled_E)

    # 실제 값과 예측 값 비교를 위해 실제 2023년 p_era 값도 복원
    y_test_actual = data_23[target].values

    # RMSE 계산
    rmse_E = np.sqrt(mean_squared_error(y_test_actual, y_pred_E))
    rmse_list.append(rmse_E)

    # MAE 계산
    mae_E = mean_absolute_error(y_test_actual, y_pred_E)
    mae_list.append(mae_E)
    
    # MAPE 계산
    mape_E = mean_absolute_percentage_error(y_test_actual, y_pred_E)
    mape_list.append(mape_E)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_E}, MAE: {mae_E}, MAPE: {mape_E}')

# 평균 RMSE, MAE, MAPE 계산
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
3/3 [==============================] - 13s 1s/step - loss: 0.0853 - accuracy: 0.0000e+00 - val_loss: 0.0252 - val_accuracy: 0.0222
Epoch 2/500
3/3 [==============================] - 0s 33ms/step - loss: 0.0339 - accuracy: 0.0000e+00 - val_loss: 0.0389 - val_accuracy: 0.0222
Epoch 3/500
3/3 [==============================] - 0s 37ms/step - loss: 0.0336 - accuracy: 0.0000e+00 - val_loss: 0.0227 - val_accuracy: 0.0222
Epoch 4/500
3/3 [==============================] - 0s 34ms/step - loss: 0.0270 - accuracy: 0.0000e+00 - val_loss: 0.0289 - val_accuracy: 0.0222
Epoch 5/500
3/3 [==============================] - 0s 38ms/step - loss: 0.0326 - accuracy: 0.0000e+00 - val_loss: 0.0281 - val_accuracy: 0.0222
Epoch 6/500
3/3 [==============================] - 0s 35ms/step - loss: 0.0319 - accuracy: 0.0000e+00 - val_loss: 0.0229 - val_accuracy: 0.0222
Epoch 7/500
3/3 [==============================] - 0s 35ms/step - loss: 0.0260 - accuracy: 0.0000e+00 - val_loss: 0.0224 - val_accuracy: 

In [7]:
y_pred_A_list = [item[0] for item in y_pred_A]

# 출력 옵션 설정
pd.set_option('display.max_rows', None)

df = pd.DataFrame({'Actual': data_23[target], 'Predicted': y_pred_A_list})
df['diff'] = df['Predicted'] - df['Actual']
print(df.sort_values(by='Actual'))

      Actual  Predicted      diff
3275    2.25   2.945331  0.695331
3276    2.28   3.287447  1.007447
3165    2.46   3.718287  1.258287
3291    2.55   3.103389  0.553389
3192    2.63   2.279509 -0.350491
3196    2.79   3.052670  0.262670
3272    3.05   3.177357  0.127357
3235    3.16   2.814539 -0.345461
3403    3.20   3.754248  0.554248
3295    3.22   3.226676  0.006676
3414    3.25   3.118687 -0.131313
3324    3.34   3.174156 -0.165844
3499    3.39   2.986092 -0.403908
3459    3.45   2.677259 -0.772741
3380    3.46   3.347084 -0.112916
3489    3.47   3.096517 -0.373483
3309    3.50   2.963012 -0.536988
3288    3.57   4.317373  0.747373
3476    3.58   4.631595  1.051595
3263    3.60   3.099762 -0.500238
3211    3.61   3.074610 -0.535390
3195    3.63   3.616450 -0.013550
3157    3.64   3.713962  0.073962
3312    3.65   3.267688 -0.382312
3345    3.66   3.116226 -0.543774
3198    3.74   4.182647  0.442647
3159    3.77   3.426696 -0.343304
3507    3.80   3.977421  0.177421
3243    3.86  